In [ ]:
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import json
import os
from tqdm import tqdm

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, Flatten, BatchNormalization, Conv2D, MaxPool2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.layers import AveragePooling2D, MaxPooling2D, GlobalAveragePooling2D, LeakyReLU
from keras.models import Model
from keras.regularizers import l2

In [ ]:
from typing import Tuple

def resize_with_pad(image: np.array,
                    new_shape: Tuple[int, int],
                    padding_color: Tuple[int] = (255, 255, 255)) -> np.array:
    original_shape = (image.shape[1], image.shape[0])
    ratio = float(max(new_shape))/max(original_shape)
    new_size = tuple([int(x*ratio) for x in original_shape])
    image = cv2.resize(image, new_size)
    delta_w = new_shape[0] - new_size[0]
    delta_h = new_shape[1] - new_size[1]
    top, bottom = delta_h//2, delta_h-(delta_h//2)
    left, right = delta_w//2, delta_w-(delta_w//2)
    image = cv2.copyMakeBorder(image, top, bottom, left, right, cv2.BORDER_CONSTANT, value=padding_color)
    return image

In [ ]:
labels_t = pd.read_csv("train_labels.csv")
train_dir= "/content/datathon/train/train/"

In [ ]:
X_tr = []
Y_tr = []

for name in labels_t['id'].values:
    temp = cv2.imread(train_dir + str(name)+".tif")
    temp = cv2.cvtColor(temp,cv2.COLOR_BGR2YUV)
    temp[:,:,0] = cv2.equalizeHist(temp[:,:,0])
    temp = cv2.cvtColor(temp, cv2.COLOR_YUV2BGR)
    temp = resize_with_pad(temp, (256, 256))
    temp = cv2.cvtColor(temp, cv2.COLOR_BGR2GRAY)
    temp = np.expand_dims(temp, axis=2)
    X_tr.append(temp)

X_tr = np.array(X_tr)
X_tr = X_tr.astype('float32')

In [ ]:
X_tr/=255
X_tr.shape

In [ ]:
y_train = labels_t['label']
y_train=np.array(y_train)
y_train.shape

In [ ]:
model = Sequential([
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding = 'same', input_shape=(256,256,1),kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01)),
    MaxPool2D(pool_size=(2, 2), strides=2),
    Dropout(0.2),
    BatchNormalization(momentum=0.9, epsilon=1e-5, gamma_initializer="uniform"),
    Conv2D(filters=64, kernel_size=(5, 5), activation='relu', padding = 'same',kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01)),
    MaxPool2D(pool_size=(2, 2), strides=2),
    Dropout(0.2),
    BatchNormalization(momentum=0.9, epsilon=1e-5, gamma_initializer="uniform"),
    Conv2D(filters=64, kernel_size=(5, 5), activation='relu', padding = 'same',kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01)),
    MaxPool2D(pool_size=(2, 2), strides=1, padding="same"),
    BatchNormalization(momentum=0.9, epsilon=1e-5, gamma_initializer="uniform"),
    Dropout(0.2),
    Conv2D(filters=128, kernel_size=(7, 7), activation='relu', padding = 'same',kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01)),
    MaxPool2D(pool_size=(2, 2), strides=2),
    BatchNormalization(momentum=0.9, epsilon=1e-5, gamma_initializer="uniform"),
    Conv2D(filters=128, kernel_size=(7, 7), activation='relu', padding = 'same',kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01)),
    MaxPool2D(pool_size=(2, 2), strides=2),
    BatchNormalization(momentum=0.9, epsilon=1e-5, gamma_initializer="uniform"),
    Flatten(),
    Dropout(0.2),
    Dense(units=32, activation='relu'),
    Dropout(0.2),
    Dense(16, activation='softmax'),
])

In [ ]:
model.compile(optimizer=Adam(learning_rate=0.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
! mkdir model_checkpoints

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
checkpoint_path = '/content/model_checkpoints/weights.best.hdf5'
checkpoint = ModelCheckpoint(
    filepath=checkpoint_path,
    monitor='val_accuracy',
    save_freq='epoch',
    save_best_only=True,
    mode='max',
    verbose=1
)

In [ ]:
from keras.callbacks import ReduceLROnPlateau
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=2, min_lr=0.00001, verbose=1)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
early_stop=EarlyStopping(monitor='val_loss',mode='min',verbose=1,patience=2)

In [ ]:
history = model.fit(X_tr,y_train,
              batch_size= 32,
              epochs=50,
              validation_split=0.2,
              callbacks=[early_stop,checkpoint,reduce_lr],
              shuffle=True,
              verbose=1)

In [ ]:
model_json = model.to_json()
with open('model.json', 'w') as json_file:
  json_file.write(model_json)
model.save_weights('/content/model_checkpoints/weights.best.hdf5')

In [ ]:
loss_train = history.history['loss']
loss_val = history.history['val_loss']

plt.plot(loss_train, 'g', label='Training loss')
plt.plot(loss_val, 'b', label='validation loss')
plt.title('Training and Validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
train = history.history['accuracy']
val = history.history['val_accuracy']
plt.plot(train, 'g', label='Training accuracy')
plt.plot(val, 'b', label='Validation accuracy')
plt.title('Training and Validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()